In [ ]:
# Mount google drive

from google.colab import drive

drive.mount('/content/drive')

In [ ]:
!rm -r data/
!rm -r __MACOSX/
!rm -r test/
!rm -r train/
!rm -r valid/
!rm data.*
!rm README.*

In [ ]:
!mkdir data
!cp drive/MyDrive/FYP/10000_dataset_binary/data.zip ./data/data.zip

In [ ]:
!unzip ./data/data.zip -d data/

In [ ]:
import yaml
import os

# Modify original data label
def update_original_data(yaml_path, labels_paths):
    yaml_data = {}
    with open(yaml_path, 'r') as f:
        yaml_data = yaml.safe_load(f)
        f.close()

    for path in labels_paths:
        for file in os.listdir(path):
            new_content = []
            # Read label files
            with open(f'{path}/{file}', 'r') as f:
                content = f.readlines()
                for line in content:
                    # update the class id to 0, as only 1 class
                    words = line.split(' ')[1:]
                    new_words = ['0'] + words
                    new_line = ' '.join(new_words)
                    new_content.append(new_line)


            # Write label files
            with open(f'{path}/{file}', 'w') as f:
                f.writelines(new_content)
                f.close()

# Update data/train/labels
labels_paths = ['data/train/labels', 'data/valid/labels']
update_original_data('data/data.yaml', labels_paths)

In [ ]:
from os.path import split
import os

# Filter out malformatted files
# For yolo without OBB, each line in label file contains only 5 values
def malformat_filter(paths):
    for path in paths:
        targets = []
        for file in os.listdir(path+'labels'):
            lines = []
            label_path = path+'labels/'+file
            with open(label_path, 'r') as f:
                lines = f.readlines()

            for line in lines:
                words = line.split(' ')
                if len(words) > 5:
                    targets.append('.'.join(label_path.split('.')[:-1]))
                    break

        for target in targets:
            os.remove(target+'.txt')
            os.remove(target.replace('labels', 'images', 1)+'.jpg')

malformat_filter(['data/train/', 'data/valid/', 'data/test/'])

In [ ]:
# Modify original yaml to accomodate Hong Kong Mahjong Tiles
import yaml
import os

def update_original_yaml(path):
    yaml_content = {}
    with open(path, 'r') as f:
        yaml_content = yaml.safe_load(f)
        print(yaml_content)
        f.close()

    yaml_content['path'] = './data'
    yaml_content['test'] = 'test/images'
    yaml_content['train'] = 'train/images'
    yaml_content['val'] = 'valid/images'
    yaml_content['names'] = ['tile']
    yaml_content['nc'] = len(yaml_content['names'])

    with open('yolo11.yaml', 'w') as f:
        yaml.safe_dump(yaml_content, f)

update_original_yaml('./data/data.yaml')

In [ ]:
!pip install ultralytics

In [ ]:
# @title
from ultralytics import YOLO

model = YOLO('yolo11m.pt')
# model.reset_weights()


result = model.train(
    data='./yolo11.yaml',
    epochs=50,
    imgsz=1088,
    pretrained=False,
    patience=15,
    device=0,
    optimizer='AdamW'
)

In [ ]:
model.save('MJ_tile_detection_yolo11.pt')

# Task
Archive the YOLOv8 model training results, including the `runs` directory, into a timestamped zip file and upload it to your Google Drive at `"/content/drive/MyDrive/"`.

## Create Timestamped Folder

### Subtask:
Create a new folder named with the current date and time.


**Reasoning**:
To create a folder with the current date and time as its name, I need to import the `datetime` module to get the current timestamp, format it into a string, and then use the `os` module to create the directory.



In [ ]:
!zip -r binary_runs.zip runs/
!cp binary_runs.zip /content/drive/MyDrive/FYP/10000_dataset_binary/
!cp MJ_tile_detection_yolo11.pt /content/drive/MyDrive/FYP/10000_dataset_binary/